# Project: Food Vision Big TM

In [1]:
# Import libraries and checking using GPU
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile

# Check if GPU is available
gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:   
    print(f"GPU is available: {gpu_available}")
else:
    print("GPU is not available, using CPU instead.")

2025-07-05 15:06:43.660169: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-05 15:06:43.909866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751702804.006124   15756 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751702804.033124   15756 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751702804.242053   15756 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

GPU is not available, using CPU instead.


2025-07-05 15:06:49.106635: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
#!nvidia-smi

In Colab, they provide a GPU for free, so we can use it to train our model. But in this project, I decided to use my own GPU to train the model. The GPU is a NVIDIA GeForce RTX 4050.

And I have to mearsure that it's compatible with using mixed precision training...

## Get the helper functions


In [ ]:
#!cd help_funcs && wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-07-05 15:15:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10,01K  --.-KB/s    in 0s      

2025-07-05 15:15:28 (147 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
# Import the helper functions
import sys
sys.path.append('help_funcs/')
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys, unzip_data

## Import the dataset


In [ ]:
import tensorflow_datasets as tfds
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

2025-07-05 15:20:04.107866: W external/local_xla/xla/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Could not resolve hostname', error details: Could not resolve host: metadata.google.internal".


True


In [10]:
# 
(train_data, test_data), ds_info = tfds.load(
    name="food101",
    split=["train", "validation"],
    shuffle_files=True,
    data_dir="datasets", # where to save the dataset
    with_info=True,
    as_supervised=True # data gets returned as tuple (image, label)
)

/home/dikhangcshcmut/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [15:40<?, ? url/s]
Extraction completed...: 0 file [15:40, ? file/s]
Dl Completed...:   0%|          | 0/1 [15:40<?, ? url/s]


KeyboardInterrupt: 